In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage import io, color, transform
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.util import random_noise

# Constants
image_size = (256, 256)
num_classes = 2  # Number of classes

# Load and preprocess the image dataset
def load_dataset(data_dir):
    X = []
    y = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        for image_file in os.listdir(class_dir):
            if image_file.endswith(".JPG"):
                img = io.imread(os.path.join(class_dir, image_file))
                img = transform.resize(img, image_size)
                img = color.rgb2gray(img)
                X.append(img.flatten())
                y.append(class_name)

    X = np.array(X)
    y = np.array(y)

    # Normalize pixel values to [0, 1]
    X = X / 255.0

    # Encode labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    return X, y




In [2]:
# Load the dataset
data_dir = r'C:\Users\HAI\Desktop\project\New folder'
X, y = load_dataset(data_dir)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:

# Data Augmentation
augmented_X = []
augmented_y = []

for i in range(len(X_train)):
    augmented_X.append(X_train[i])
    augmented_y.append(y_train[i])

    # Add Gaussian noise to the image
    noisy_image = random_noise(X_train[i], var=0.01)
    augmented_X.append(noisy_image)
    augmented_y.append(y_train[i])

    noisy_image = random_noise(X_train[i], mode='s&p', amount=0.01)
    augmented_X.append(noisy_image)
    augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)

# SVM Model
svm_model = SVC(C=1.0, kernel='rbf', probability=True)  # You can adjust C and kernel as needed
svm_model.fit(augmented_X, augmented_y)



In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM Model with Hyperparameter Tuning
param_grid_svm = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid_search_svm = GridSearchCV(SVC(probability=True), param_grid_svm, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1)
grid_search_svm.fit(X_train_scaled, y_train)

best_C_svm = grid_search_svm.best_params_['C']
best_kernel_svm = grid_search_svm.best_params_['kernel']
print(f"Best C: {best_C_svm}, Best Kernel: {best_kernel_svm}")

Best C: 10, Best Kernel: rbf


In [12]:
svm_model = SVC(C=best_C_svm, kernel=best_kernel_svm, probability=True)
svm_model.fit(X_train_scaled, y_train)

# Evaluate the SVM model
y_pred_svm = svm_model.predict(X_test_scaled)
test_acc_svm = accuracy_score(y_test, y_pred_svm)
print(f"Test accuracy for SVM: {test_acc_svm * 100:.2f}%")

Test accuracy for SVM: 69.66%
